In [28]:
#This script is an attempt to create a 4-element windkessel model for BP simulation

#Author = Joshua Van Tuyl
#Team VitalCentric

#Type = Healthy

import numpy as np
import math
import matplotlib.pyplot as plt
import scipy as sp
from scipy.integrate import odeint
from scipy.integrate import solve_ivp



In [55]:
#STEP ONE - Defining Constants
#I'm using integer 1 as a placeholder until I find real values

#Total Peripheral Resistance: Fluid resistance due to periphery 
# mmHg.s.cm^-3
R = 0.79 

#Compliance of veins: Measure of how well the veins stretch during systole and diastole
# cm^3.mmHg^-1.s^2.cm^-3
C = 1.22

#Characteristic impedance: Resistance due to aortic and pulmonary valve
# mmHg.s.cm^-3
r = 0.056

#Inertance: Measure of the blood fluid inertia
# mmHg.s^2.cm^-3
L = 0.0051

In [56]:
#STEP TWO - Defining ODEs
#The 4 element wk model are two coupled linear odes
#Need to truncate into one function



#Also need to define input signal
def q(t):
    if t >= 0 and t <= t+0.3:
        return 500 * (math.sin((3.145*t)/0.3))**2
    elif t > t+0.3 and t < t+0.8:
        return 0

def dSdt(t, S):
    P, q = S
    return [(-1/(R*C))*P + (1/C)*q,
           (-r/L)*q + (r/L)*q]
P_0 = 80
q_0 = 500
S_0 = (P_0, q_0)

In [57]:
t = np.linspace(0, 100, 100)
soln = odeint(dSdt, y0=S_0, t=t, tfirst = True)

In [58]:
P_soln = soln.T[0]

In [59]:
q_soln = []
for t in range(0,100):
    i = q(t)
    q_soln.append(i)